<a href="https://colab.research.google.com/github/netq-oist/estimation-private/blob/main/Bell_diagonal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble
Given a state of the Bell diagonal form $\rho = p_1|\Phi^+\rangle\langle \Phi^+| + p_2|\Psi^+\rangle\langle \Psi^+|+ p_3|\Phi^-\rangle\langle \Phi^-|+ p_4|\Psi^-\rangle\langle \Psi^-|$, the aim is to estimate the value of $p_1,p_2,p_3,p_4$. We consider, estimation of $p_1,p_2,p_3,p_4$ from the measurement statistics of a distillation protocol. The distillation protocol is as follows.

1) Alice and Bob start with two copies of a noisy Bell diagonal state.

2) Each applies local XOR operations on the local parts of their two copies, with one copy as the control and another one as the target.

3) Each of them measures the target qubit in the $Z$-basis and communicates the results classically.

4) If they obtain correlated outcomes i.e, $00,11$ they keep the unmeasured copy, otherwise they discard the state.

In this way they obtain $p_{00}$.

In a similar way, they do another (variation of) protocol,

1) Alice and Bob start with two copies of a noisy Bell diagonal state.

2) Each applies local XOR operations on the local parts of their two copies, with one copy as the control and another one as the target.

3) This time each of them measures the control qubit in the $X$-basis and communicates the results classically.

4) If they obtain correlated outcomes i.e, $++,--$ they keep the unmeasured copy, otherwise they discard the state.

In this way they obtain $p_{++}$.

They now do another (variation of) protocol which is exactly same as the first one but this time they apply bilateral $\pi/2$ rotaion on each of the four qubit initially. This implies, they start the distillation protocol with a rotaed bell diagonal state of the form $\rho = p_2|\Phi^+\rangle\langle \Phi^+| + p_1|\Psi^+\rangle\langle \Psi^+|+ p_3|\Phi^-\rangle\langle \Phi^-|+ p_4|\Psi^-\rangle\langle \Psi^-|$.

From this protocol, they obtain another probability $p_{HH}$.

From this $p_{00}, p_{++}, p_{HH}$, our aim is to invert $p_1,p_2,p_3,p_4$ even in presence of noise.

In [ ]:
# Initialisation

import numpy as np
import math as mt
import numpy.linalg as nl
import functools
import random

import matplotlib.pyplot as plt

def multikron(arrays):
  return functools.reduce(np.kron, arrays)

# up-down states for Z basis
upZ = np.matrix([[1.0],[0.0]])
downZ = np.matrix([[0.0],[1.0]])

# up-down states for X basis
upX = (1./mt.sqrt(2.0))*(upZ + downZ)
downX = (1./mt.sqrt(2.0))*(upZ - downZ)

# up-down states for Y basis
upY = (1./mt.sqrt(2.0))*(upZ + 1j* downZ)
downY = (1./mt.sqrt(2.0))*(upZ - 1j* downZ)

# Pauli matrices
s0 = np.matrix([[1.0, 0.0],[0.0, 1.0]])      # I_2
s1 = np.matrix([[0.0, 1.0],[1.0, 0.0]])      # Pauli-X
s2 = np.matrix([[0.0, -1j],[1j, 0.0]])       # Pauli-Y
s3 = np.matrix([[1.0, 0.0],[0.0, -1.0]])     # Pauli-Z

# Bell states
phip = (1./mt.sqrt(2.0)) * np.matrix([ [1.], [0.], [0.], [1.]])
phim = (1./mt.sqrt(2.0)) * np.matrix([ [1.], [0.], [0.], [-1.]])
psip = (1./mt.sqrt(2.0)) * np.matrix([ [0.], [1.], [1.], [0.]])
psim = (1./mt.sqrt(2.0)) * np.matrix([ [0.], [1.], [-1.], [0.]])

# Gates
CNOT = np.array([[1.0,0.0,0.0,0.0],
                 [0.0,1.0,0.0,0.0],
                 [0.0,0.0,0.0,1.0],
                 [0.0,0.0,1.0,0.0]]) #CNOT gate

SWAP = np.array([[1.0,0.0,0.0,0.0],
                 [0.0,0.0,1.0,0.0],
                 [0.0,1.0,0.0,0.0],
                 [0.0,0.0,0.0,1.0]])  #SWAP gate

ID = np.array([[1.0,0.0,0.0,0.0],
                 [0.0,1.0,0.0,0.0],
                 [0.0,0.0,1.0,0.0],
                 [0.0,0.0,0.0,1.0]]) # 4X4 Identity matrix

#HAD = (1./mt.sqrt(2.0)) * np.matrix([[1.0, 1.0],[1.0, -1.0]]) #Hadamard gate

#ZROT = np.matrix([[1.0, 0.0],[0.0, 1j]]) #Bilateral Z rotaion

XROT = (1./2.) * np.matrix([[1.0 + 1j, 1.0 - 1j],[1.0 - 1j, 1.0 + 1j]]) #Bilateral X rotaion

# Action of noise and calculation of probability

Given Bell diagonal state: $\rho = p_1|\Phi^+\rangle\langle \Phi^+| + p_2|\Psi^+\rangle\langle \Psi^+|+ p_3|\Phi^-\rangle\langle \Phi^-|+ p_4|\Psi^-\rangle\langle \Psi^-|$

Dephasing parameter $p = \frac{1}{2}(1-e^{-\frac{t}{T_2}})$ and depolarizing parameter $q = (1-e^{-\frac{t}{T_1}})$

Action of dephasing operation:
$\Lambda_{deph} (\rho) = (1-p)\rho + p \sigma_Z \rho \sigma_Z$

Action of depolarizing operation:
$\Lambda_{depol} (\rho) = (1-q)\rho + \frac{q}{2} \mathbb{I}$

Final state after the action of these two noise
$\Lambda_{depol}(\Lambda_{deph}(\rho_w))$

Action of noisy CNOT on $\rho_{A_1A_2B_1B_2}$:

\begin{align}
\Lambda_{\text{CNOT}}(\rho_{A_1 A_2 B_1 B_2})
=\quad &(1-y_1)(1-y_2) \,\text{CNOT}_{A_1 A_2}\otimes \text{CNOT}_{B_1 B_2} \,  \rho_{A_1A_2B_1B_2} \,\text{CNOT}_{A_1 A_2} \otimes \text{CNOT}_{B_1 B_2}\\
+ &\frac{y_2(1-y_1)}{4} \,\text{CNOT}_{A_1 A_2}\otimes I_{B_1 B_2} \,  \rho_{A_1A_2B_1B_2} \,\text{CNOT}_{A_1 A_2} \otimes I_{B_1 B_2}\\
+ &\frac{y_1(1-y_2)}{4}\,I_{A_1 A_2}\otimes\text{CNOT}_{B_1 B_2} \,  \rho_{A_1A_2B_1B_2} \,I_{A_1 A_2}\otimes\text{CNOT}_{B_1 B_2}\\
+ &\frac{y_1 y_2}{16} \mathbb{I}
\end{align}

Final state after all the noise acion:
\begin{equation}
\rho_{final} = \Lambda_{\text{CNOT}}((\mathbb{I} \otimes SWAP \otimes \mathbb{I})(\Lambda_{depol}(\Lambda_{deph}(\rho)) \otimes \rho)(\mathbb{I} \otimes SWAP \otimes \mathbb{I}))
\end{equation}

Noisy measurement:

$M_{pz}= pz |0\rangle\langle 0|+ (1-pz)|1\rangle\langle 1|$
Probability of getting 00 outcome is given by Born rule
\begin{equation}
p_{00} = Tr[ \langle M_{p_{A_z}}|_{A_2} \langle M_{p_{B_z}}|_{B_2} (\rho_{final}) |M_{p_{A_z}}⟩_{A_2} |M_{p_{B_z}}⟩_{B_2} ]
\end{equation}

$M_{p_x}= p_x |+\rangle\langle +|+ (1-p_x)|1\rangle\langle 1|$
Probability of getting ++ outcome is given by Born rule
\begin{equation}
p_{++} = Tr[ \langle M_{p_{A_x}}|_{A_1} \langle M_{p_{B_x}}|_{B_1} (\rho_{final}) |M_{p_{A_x}}⟩_{A_1} |M_{p_{B_x}}⟩_{B_1} ]
\end{equation}

# Calculation in terms of Bell vector
Bell diagonal state: $\rho = p_1 |\Phi^+\rangle\langle \Phi^+| + p_2 |\Psi^+\rangle\langle \Psi^+| + p_3 |\Phi^-\rangle\langle \Phi^-| + p_4 |\Psi^-\rangle\langle \Psi^-|$ denoted as $(p_1,p_2,p_3,p_4)$.

Rotated Bell diagonal state: $\rho = p_2 |\Phi^+\rangle\langle \Phi^+| + p_1 |\Psi^+\rangle\langle \Psi^+| + p_3 |\Phi^-\rangle\langle \Phi^-| + p_4 |\Psi^-\rangle\langle \Psi^-|$ denoted as $(p_2,p_1,p_3,p_4)$.

Action of X operator: $(\mathbb{I} \otimes X)(p_1, p_2, p_3, p_4) ⇒ (p_2, p_1, p_4, p_3)$

Action of Z operator: $(\mathbb{I} \otimes Z)(p_1, p_2, p_3, p_4) ⇒ (p_3, p_4, p_1, p_2)$

Action of Y operator: $(\mathbb{I} \otimes Y)(p_1, p_2, p_3, p_4) ⇒ (p_4, p_3, p_2, p_1)$

Therefore, under dephasing: $(\Lambda_{deph})(p_1, p_2, p_3, p_4) ⇒ ((1-p_A)(1-p_B) + p_Ap_B)(p_1, p_2, p_3, p_4) + (p_A(1-p_B) + p_B(1-p_A))(p_3, p_4, p_1, p_2)$.

Under depolarising: $(\Lambda_{depol})(p_1, p_2, p_3, p_4) ⇒ (1-3q_A/4)(1-3q_B/4)(p_1, p_2, p_3, p_4)  + (q_Aq_B/16)(2(1, 1, 1, 1) + (p_1, p_2, p_3, p_4)) + ((1-3q_A/4)q_B/4 +(1-3q_B/4)q_A/4)((p_2, p_1, p_4, p_3)+(p_3, p_4, p_1, p_2)+(p_4, p_3, p_2, p_1))$.

Under CNOT: $p_1p_1' → p_1p_1', p_1p_2' → p_1p_2', p_1p_3' → p_3p_3', p_1p_4' → p_3p_4', p_2p_1' → p_2p_2', p_2p_2' → p_2p_1', p_2p_3' → p_4p_4', p_2p_4' → p_4p_3',   p_3p_1' → p_3p_1', p_3p_2' → p_3p_2', p_3p_3' → p_1p_3', p_3p_4' → p_1p_4', p_4p_1' → p_4p_2', p_4p_2' → p_4p_1', p_4p_3' → p_2p_4', p_4p_4' → p_2p_3'$ where the coefficients corresponding to the second copy are denoted by prime.

Under noisy measurement:

Noisy Z measurement: with probability $p_{mA}p_{mB}$ and $(1-p_{mA})(1-p_{mB})$ calculate the coefficient of $|\Phi^+\rangle$ and $|\Phi^-\rangle$ and with probability $(1-p_{mA})p_{mB}$ and $p_{mA}(1-p_{mB})$ calculate the coefficient of $|\Psi^+\rangle$ and $|\Psi^-\rangle$.

Noisy X measurement: with probability $p_{mA}p_{mB}$ and $(1-p_{mA})(1-p_{mB})$ calculate the coefficient of $|\Phi^+\rangle$ and $|\Psi^+\rangle$ and with probability $(1-p_{mA})p_{mB}$ and $p_{mA}(1-p_{mB})$ calculate the coefficient of $|\Phi^-\rangle$ and $|\Psi^-\rangle$.

In [ ]:
def BellVector(p1, p2, p3, p4):
  return [p1, p2, p3, p4]

# Bell diagonal state
def getBellDiagonalState(p1,p2,p3,p4):
  return BellVector(p1, p2, p3, p4)

def getRotatedBellDiagonalState(p1,p2,p3,p4):
  return BellVector(p2, p1, p3, p4)
# Werner state
def getWernerState(w):
  return BellVector(1.-3.*w/4., w/4., w/4., w/4.)

# Apply X operation on Bell diagonal state
def applyXtoBellVector(vector):
  vectornew = [vector[1],vector[0],vector[3],vector[2]]
  return vectornew

# Apply Z operation on Bell diagonal state
def applyZtoBellVector(vector):
  vectornew = [vector[2],vector[3],vector[0],vector[1]]
  return vectornew

# Apply Y operation on Bell diagonal state
def applyYtoBellVector(vector):
  vectornew = [vector[3],vector[2],vector[1],vector[0]]
  return vectornew

# Dephasing parameter
def getDephasingParameter(t,T2):
  return 0.5*(1. - mt.exp(-t/T2))

# Deploarizing parameter
def getDepolarizingParameter(t,T1):
  return 1. - mt.exp(-t/T1)

# Dephasing function with Alice's parameter pA and Bob's parameter pB
def applyDephasing(inputVector,t,T2):
  pA = getDephasingParameter(t,T2) #Alice's Dephasing parameter
  pB = getDephasingParameter(t,T2) #Bob's Dephasing parameter
  return list(((1-pA)*(1-pB) + pA*pB)* np.array(inputVector) + ((1-pA)*pB + pA*(1-pB))* np.array(applyZtoBellVector(inputVector)))

# Depolarizing function with Alice's parameter qA and Bob's parameter qB
def applyDepolarizing(inputVector,t,T1):
  qA = getDepolarizingParameter(t,T1) #Alice's Depolarizing parameter
  qB = getDepolarizingParameter(t,T1) #Bob's Depolarizing parameter
  return list((1.-3.*qA/4.)*(1.-3.*qB/4.)* np.array(inputVector) + \
   (qA*qB/16)* (2*np.array([1., 1., 1., 1.]) + np.array(inputVector)) + ((1-3*qA/4)*(qB/4) + \
    (1-3*qB/4)*(qA/4))*(np.array(applyXtoBellVector(inputVector)) + \
    np.array(applyYtoBellVector(inputVector)) + np.array(applyZtoBellVector(inputVector))))

# Expression for general state after the action of dephasing and deploarizing noise
def FinalStateAfterActionOfNoise(inputState,t,T1,T2):
  return applyDepolarizing(applyDephasing(inputState,t,T2),t,T1)

# Expression of two copy state rhoA1B1A2B2 where firstcopy A1B1 will undergo noise for time t and the second copy is noise free
def rhoA1B1A2B2(inputState,t,T1,T2):
  return np.kron(applyDepolarizing(applyDephasing(inputState,t,T2),t,T1),inputState)

# State after the action of noisy CNOT@CNOT
def applyCNOT1(inputMatrix,y1,y2):
  inputMatrixNew = [inputMatrix[0], inputMatrix[1], inputMatrix[10], \
  inputMatrix[11], inputMatrix[5], inputMatrix[4], inputMatrix[15], \
  inputMatrix[14], inputMatrix[8], inputMatrix[9], inputMatrix[2], \
  inputMatrix[3], inputMatrix[13], inputMatrix[12], inputMatrix[7], inputMatrix[6]]
  return list((1-y1)*(1-y2)* np.array(inputMatrixNew) + (y1*y2/16.+(1-y1)*y2/4.+(1-y2)*y1/4.) *np.kron([1.,1.,1.,1.], [1.,1.,1.,1.]))

# Effect of Noisy Z measurement with probability p up projector and probability (1-p) down projector on A2B2 part
def NoisyMeasurementZBasis(inputVector, mA, mB):
  return (mA*mB + (1.-mA)*(1.-mB))*(1./2.)*(inputVector[0] + inputVector[2] + \
      inputVector[4] + inputVector[6] + inputVector[8] + inputVector[10] + \
      inputVector[12] + inputVector[14]) + ((1.-mA)*mB + mA*(1.-mB))*(1./2.)* \
      (inputVector[1] + inputVector[3] + inputVector[5] + inputVector[7] + \
      inputVector[9] + inputVector[11] + inputVector[13] + inputVector[15])

# Effect of Noisy X measurement with probability p up projector and probability (1-p) down projector on A1B1 part
def NoisyMeasurementXBasis(inputVector, mA, mB):
  return (mA*mB + (1.-mA)*(1.-mB))*(1./2.)*(inputVector[0] + inputVector[1] + \
      inputVector[2] + inputVector[3] + inputVector[4] + inputVector[5] + \
      inputVector[6] + inputVector[7]) + ((1.-mA)*mB + mA*(1.-mB))*(1./2.)* \
      (inputVector[8] + inputVector[9] + inputVector[10] + inputVector[11] + \
      inputVector[12] + inputVector[13] + inputVector[14] + inputVector[15])
#print(NoisyMeasurementXBasis(np.kron([0,0,0,1],[0,0,0,1]), 0, 0))
# Calculation of p001 for Bell diagonal state when Alice and Bob are measuring their second copy on Z basis i.e, on A2B2 part
def p001(p1,p2,p3,p4,t,T1,T2,y1,y2,mZA,mZB):
  return NoisyMeasurementZBasis(applyCNOT1(rhoA1B1A2B2(getBellDiagonalState(p1,p2,p3,p4),t,T1,T2),y1,y2),mZA,mZB)

# Calculation of p002 for Bell diagonal state when Alice and Bob are measuring their first copy on X basis i.e, on A2B2 part
def p002(p1,p2,p3,p4,t,T1,T2,y1,y2,mXA,mXB):
  return NoisyMeasurementXBasis(applyCNOT1(rhoA1B1A2B2(getBellDiagonalState(p1,p2,p3,p4),t,T1,T2),y1,y2),mXA,mXB)

#Calculation for p003 for Bell diagonal state when Alice and Bob are measuring their second copy on Z basis on the Bilateral X-rotated Bell state
def p003(p1,p2,p3,p4,t,T1,T2,y1,y2,mZA,mZB):
  return NoisyMeasurementZBasis(applyCNOT1(rhoA1B1A2B2(getRotatedBellDiagonalState(p1,p2,p3,p4),t,T1,T2),y1,y2),mZA,mZB)

0.4546826882694955
0.41758001150890983
0.4546826882694955


# Place all input variables here

For simulation of experiment

In [ ]:
#Bell parameters
p1exp = 1.
p2exp = 0.
p3exp = 0
p4exp = 0.

# Some useful numbers
eps = 1e-5
epsilon = 0.01

# Parameter related to dephasing and depolarizing noise
T1 = 50.0
T2 = 50.0

# Initial parameter for noisy CNOT
y1 = 0.
y2 = 0.

#Noisy measurement
mZA = 1.0
mZB = 1.0
mXA = 1.0
mXB = 1.0
mYA = 1.0
mYB = 1.0

# Expeimental parameters
Nsamples = 5
pSuccess = 0.1
#p00exp = ncount/Nsamples
#tave = 1


# Simulation of experiment for evaluating p00exp

After the first copy has been generated one need to wait (say) $t$ times for the second copy. And this $t$ will follow some geometric distribution with success probability $p_{success}$. If Nsamples is the number of times the distillation experment goes on, then we can calculate the probability of obtaining outcome $00, ++, HH$ by the following simulation.

In [ ]:
# This is new definition of p00 to get rid of other noise parameters
def p001Expected(p1,p2,p3,p4,t):
  return p001(p1,p2,p3,p4,t,T1,T2,y1,y2,mZA,mZB) # Z basis

def p002Expected(p1,p2,p3,p4,t):
  return p002(p1,p2,p3,p4,t,T1,T2,y1,y2,mXA,mXB) # X basis

def p003Expected(p1,p2,p3,p4,t):
  return p003(p1,p2,p3,p4,t,T1,T2,y1,y2,mZA,mZB) # Rotated Bell state, Z basis


np.random.seed(8888)
# generate t array (geometric distribution) and x array (uniform distribution)
#tArray = np.random.geometric(pSuccess, Nsamples)
tArray=[5,5,5,5,5]

# Calculation for p001 i.e, (Bell state with Z measurement on second copy)
# Calculating p001 average which we will use for inverting p001 to Bell parameters
def p001samples(p1,p2,p3,p4, Nsamples, tArray):
  # p00 sum over i
  p001sum = 0.

  i = 0

  while i < Nsamples:
    # calculate p00 per instance i
    p001sum += p001Expected(p1,p2,p3,p4,tArray[i])

    i += 1

  return p001sum/Nsamples

# Simulation of experiment by calculating p001 by performing the experiment Nsamples time where t is choosen from geometric distribution
def p001Experimental(p1exp, p2exp, p3exp, p4exp, Nsamples, pSuccess, tArray):
  # initialize counting 00's
  Ncount = 0.
  i = 0

  while i < Nsamples:
    # we count the 00's
    # uniformly sample for some success probability
    x = random.uniform(0.,1.)

    # compare the success probabilty
    if x < p001Expected(p1exp, p2exp, p3exp, p4exp, tArray[i]):
      # count successful p00's
      Ncount +=1

    i += 1

  # calculation of p001exp
  return Ncount/Nsamples

# Calculation for p002 i.e, (Bell state with X measurement on first copy)
# Calculating p002 average which we will use for inverting p002 to Bell parameter
def p002samples(p1,p2,p3,p4, Nsamples, tArray):
  # p00 sum over i
  p00sum2 = 0.

  i = 0

  while i < Nsamples:
    # calculate p00 per instance i
    p00sum2 += p002Expected(p1,p2,p3,p4,tArray[i])

    i += 1

  return p00sum2/Nsamples

# Simulation of experiment by calculating p002 by performing the experiment Nsamples time where t is choosen from geometric distribution
def p002Experimental(p1exp, p2exp, p3exp, p4exp, Nsamples, pSuccess, tArray):
  # initialize counting 00's
  Ncount = 0.
  i = 0

  while i < Nsamples:
    # we count the 00's
    # uniformly sample for some success probability
    x = random.uniform(0.,1.)

    # compare the success probabilty
    if x < p002Expected(p1exp, p2exp, p3exp, p4exp, tArray[i]):
      # count successful p00's
      Ncount +=1

    i += 1

  # calculation of p00exp
  return Ncount/Nsamples


# Calculation for p003 i.e, (Rotated Bell state with Z measurement on second copy)
# Calculating p003 average which we will use for inverting p003 to Bell parameter
def p003samples(p1,p2,p3,p4, Nsamples, tArray):
  # p00 sum over i
  p00sum3 = 0.

  i = 0

  while i < Nsamples:
    # calculate p00 per instance i
    p00sum3 += p003Expected(p1,p2,p3,p4,tArray[i])

    i += 1

  return p00sum3/Nsamples

# Simulation of experiment by calculating p003 by performing the experiment Nsamples time where t is choosen from geometric distribution
def p003Experimental(p1exp, p2exp, p3exp, p4exp, Nsamples, pSuccess, tArray):
  # initialize counting 00's
  Ncount = 0.
  i = 0

  while i < Nsamples:
    # we count the 00's
    # uniformly sample for some success probability
    x = random.uniform(0.,1.)

    # compare the success probabilty
    if x < p003Expected(p1exp, p2exp, p3exp, p4exp, tArray[i]):
      # count successful p00's
      Ncount +=1

    i += 1

  # calculation of p00exp
  return Ncount/Nsamples

# This is just for checking if instead of smulating experiment we get actual p00 value corresponding to Bell parameters in presence of all kind of noise
def p00expn(p1exp, p2exp, p3exp, p4exp):
  i = 0
  p001expNew = 0
  p002expNew = 0
  p003expNew = 0
  while i < Nsamples:
    p001expNew += p001Expected(p1exp, p2exp, p3exp, p4exp, tArray[i])
    p002expNew += p002Expected(p1exp, p2exp, p3exp, p4exp, tArray[i])
    p003expNew += p003Expected(p1exp, p2exp, p3exp, p4exp, tArray[i])
    i = i+1
  return p001expNew/Nsamples, p002expNew/Nsamples, p003expNew/Nsamples
p001exp2 = p00expn(p1exp, p2exp, p3exp, p4exp)[0]
p002exp2 = p00expn(p1exp, p2exp, p3exp, p4exp)[1]
p003exp2 = p00expn(p1exp, p2exp, p3exp, p4exp)[2]


# Inverting $p_{00}, p_{++}, p_{HH}$ to estimate $p_1,p_2,p_3,p_4$
We use sympy method for invertion

In [ ]:
import sympy as sym
from sympy import solve
p1,p2,p3,p4 = sym.symbols('p1,p2,p3,p4', Real=True)
eq1 = sym.Eq(p001samples(p1, p2, p3, p4, Nsamples, tArray), p001exp2)
eq2 = sym.Eq(p002samples(p1, p2, p3, p4, Nsamples, tArray), p002exp2)
eq3 = sym.Eq(p003samples(p1, p2, p3, p4, Nsamples, tArray), p003exp2)
eq4 = sym.Eq(p1 + p2 + p3 + p4, 1)
result = sym.solve([eq1,eq2,eq3,eq4],(p1,p2,p3,p4))
print(result)

[(-0.500000000000000, 0.499999999999999, 0.500000000000000, 0.500000000000000), (-2.31232828134397e-16, -2.31232828134397e-16, 6.46509765908855e-16, 1.00000000000000), (-2.31232828134397e-16, -2.31232828134397e-16, 1.00000000000000, 6.46509765908855e-16), (6.46509765908855e-16, 1.00000000000000, -2.31232828134397e-16, -2.31232828134397e-16), (0.499999999999999, -0.500000000000000, 0.500000000000000, 0.500000000000000), (0.500000000000000, 0.500000000000000, -0.500000000000000, 0.499999999999999), (0.500000000000000, 0.500000000000000, 0.499999999999999, -0.500000000000000), (1.00000000000000, 6.46509765908855e-16, -2.31232828134397e-16, -2.31232828134397e-16)]


# Other methods for inversion

Using Gekko

In [ ]:
pip install gekko

In [ ]:
from gekko import GEKKO

#initialize a GEKKO model
m = GEKKO()

#add GEKKO variables
p1 = m.Var(value = 0, lb=0, ub=1)
p2 = m.Var(value = 0, lb=0, ub=1)
p3 = m.Var(value = 0, lb=0, ub=1)
p4 = m.Var(value = 0, lb=0, ub=1)


#define the constraints
m.Equation(p001samples(p1, p2, p3, p4, Nsamples, tArray) == p001exp2)
m.Equation(p002samples(p1, p2, p3, p4, Nsamples, tArray) == p002exp2)
m.Equation(p003samples(p1, p2, p3, p4, Nsamples, tArray) == p003exp2)
m.Equation(p1 + p2 + p3 + p4 == 1.)

#solve a system of non-dynamic equations
m.options.IMODE = 1
m.solve(disp=False)

print(p1.value,p2.value,p3.value,p4.value)

Using scipy

In [ ]:
import scipy.optimize as opt
from scipy.optimize import fsolve
def f(variables):
  (p1, p2, p3, p4) = variables
  first_eq = p001samples(p1, p2, p3, p4, Nsamples, tArray) - p001exp2
  second_eq = p002samples(p1, p2, p3, p4, Nsamples, tArray) - p002exp2
  third_eq = p003samples(p1, p2, p3, p4, Nsamples, tArray) - p003exp2
  fourth_eq = p1 + p2 +p3 +p4 - 1.
  return [first_eq, second_eq, third_eq, fourth_eq]

solution = opt.fsolve(f, (1,0.3,0,0) )
print(solution)

[ 9.99999990e-01  9.79249744e-09  1.02124865e-08 -1.00421239e-08]


In [ ]:
import scipy.optimize as opt
from scipy.optimize import fsolve

def myFunction(z):
  p1=z[0]
  p2=z[1]
  p3=z[2]
  p4=z[3]
  F=np.empty(4)
  F[0] = p001samples(p1, p2, p3, p4, Nsamples, tArray) - p001exp2
  F[1] = p002samples(p1, p2, p3, p4, Nsamples, tArray) - p002exp2
  F[2] = p003samples(p1, p2, p3, p4, Nsamples, tArray) - p003exp2
  F[3] = p1 + p2 +p3 +p4 - 1.
  return F
zGuess = np.array([0.99,0.0,0.5,0.5])
z = fsolve(myFunction, zGuess)
print(z)

#def f(variables):
#  (p1, p2, p3, p4) = variables
#  first_eq = p00Zsamples(p1, p2, p3, p4, Nsamples, tArray) - p00expZ2
#  second_eq = p00Xsamples(p1, p2, p3, p4, Nsamples, tArray) - p00expX2
#  third_eq = p00Ysamples(p1, p2, p3, p4, Nsamples, tArray) - p00expY2
#  fourth_eq = p1 + p2 +p3 +p4 - 1.
#  return [first_eq, second_eq, third_eq, fourth_eq]

#solution = opt.fsolve(f, (.7,0.3,0,0) )
#print(solution)

[ 1.1231204 -0.1231204 -0.1231204  0.1231204]


In [ ]:
import scipy.optimize as opt
from scipy.optimize import fsolve
from scipy.optimize import minimize

def myFunction(z):
  p1=z[0]
  p2=z[1]
  p3=z[2]
  p4=z[3]
  F=np.empty(4)
  F[0] = p001samples(p1, p2, p3, p4, Nsamples, tArray) - p001exp2
  F[1] = p002samples(p1, p2, p3, p4, Nsamples, tArray) - p002exp2
  F[2] = p003samples(p1, p2, p3, p4, Nsamples, tArray) - p003exp2
  F[3] = p1 + p2 +p3 +p4 - 1.
  return F

zGuess = np.array([0.99,0.0,0.5,0.5])
p1=[0,1]
p2=[0,1]
p3=[0,1]
p4=[0,1]
z = fsolve(myFunction, zGuess)
print(z)

[ 1.1231204 -0.1231204 -0.1231204  0.1231204]
